# Deep research Multiagent system


### Original doc:

    https://www.youtube.com/watch?v=mjPSkPLbu1s

<div style="width: 100%; height: 768px; overflow: hidden;">
  <iframe width="1024" height="768" src="https://www.youtube.com/embed/mjPSkPLbu1s?si=nrN8Y4pnHNAj-5WZ" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>
</div>

In [1]:
%%capture --no-stderr
%pip install -U --upgrade pip langgraph langchain_community langchain_anthropic langchain-tavily langchain_experimental langchain_ollama mcp langchain-mcp-adapters

In [2]:
from langfuse.langchain import CallbackHandler
from dotenv import load_dotenv
import os
# load environment variables from .env file
load_dotenv()
workfolder = os.getenv('WORKFOLDER')
mcp_file_path = os.getenv('MCP_SRV_PATH')

# Initialize Langfuse CallbackHandler for LangGraph/Langchain (tracing)
langfuse_handler = CallbackHandler() 
llm_config = {"configurable": {"thread_id": "abc123"}, "recursion_limit": 20, "callbacks": [langfuse_handler]}
 

In [3]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_tavily import TavilySearch

client = MultiServerMCPClient(
    {
        "mcp": {
            "command": "python",
            # Make sure to update to the full absolute path to your math_server.py file
            "args": [mcp_file_path],
            "transport": "stdio",
        },
        # "weather": {
        #     # make sure you start your weather server on port 8000
        #     "url": "http://localhost:8000/mcp/",
        #     "transport": "streamable_http",
        # }
    }
)
tools = await client.get_tools()
tools.append(TavilySearch())
tools


[StructuredTool(name='get_current_time', description='Returns current system time.', args_schema={'description': 'Returns current system time.', 'properties': {}, 'title': 'get_current_time', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x7a54c2203ba0>),
 StructuredTool(name='read_file', description='Read content from a file.', args_schema={'description': 'Read content from a file.', 'properties': {'file_path': {'description': 'Path to the file to read', 'title': 'File Path', 'type': 'string'}, 'encoding': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': 'utf-8', 'description': 'The encoding of the file.', 'title': 'Encoding'}, 'start': {'anyOf': [{'type': 'integer'}, {'type': 'null'}], 'default': 0, 'description': 'The start line. Default is 0', 'title': 'Start'}, 'end': {'anyOf': [{'type': 'integer'}, {'type': 'null'}], 'default': None, 'description': 'The end line. Default is None'

In [4]:
from langchain_core.tools import tool 
from langchain_ollama import ChatOllama
from langgraph.prebuilt import create_react_agent
from langchain_tavily import TavilySearch

system_prompt = """ 
You are a topic analizer. 
1. search the topic in the Internet
2. using the results generate questions that, together, cover the topic.
output format (json): {"topic": "",  "questions": [ "", ... ]}
"""
  # Create the agent
topic_analizer = create_react_agent(name="topic_analizer", model=ChatOllama(model="qwen3"), 
    tools=tools, response_format='json', prompt=system_prompt)

input_message = {"role": "user", "content": "Build a Multi-Agent Deep Research System"}

# Use the agent
for step in topic_analizer.stream(
    {"messages": [input_message]}, llm_config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Build a Multi-Agent Deep Research System
================================== Ai Message ==================================
Name: topic_analizer

<think>
Okay, the user wants to build a Multi-Agent Deep Research System. Let me start by understanding what that entails. A multi-agent system typically involves multiple autonomous agents that interact to achieve a common goal. Deep research might refer to using deep learning techniques or extensive data analysis. 

First, I need to break down the components. Maybe the user is looking for a system that combines multiple AI agents, each specialized in different tasks, working together. They might want to integrate deep learning models for data processing and analysis.

I should consider the tools available. The tavily_search function can help find relevant information on current research and existing systems. Using it with the right parameters could provide up-to

In [5]:
from langchain_tavily import TavilySearch
import json

system_prompt = """ 
You are a search assistant
1. For each question in the topic do an internet search
4. Output format json: [{ "question": "", "results": [{ "url": "", "title": "" }]}]}
"""
  # Create the agent
search_agent = create_react_agent(name="search_agent", model=ChatOllama(model="qwen3"), 
    tools=[TavilySearch()], response_format='json', prompt=system_prompt)

content = """{"topic": "How to build a Multi-Agent Deep Research System", "questions": [ 
{"question": "What is the core architecture of a Multi-Agent Deep Research System?"}, 
{"question": "What tools are essential for multi-agent research execution?"}, 
{"question": "How to implement a multi-agent research workflow?"}]}"""
input_message = {"role": "user", "content": content,}

# Use the agent
for step in search_agent.stream(
    {"messages": [input_message]}, llm_config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

{"topic": "How to build a Multi-Agent Deep Research System", "questions": [ 
{"question": "What is the core architecture of a Multi-Agent Deep Research System?"}, 
{"question": "What tools are essential for multi-agent research execution?"}, 
{"question": "How to implement a multi-agent research workflow?"}]}
================================== Ai Message ==================================
Name: search_agent

<think>
Okay, let's tackle this query about building a Multi-Agent Deep Research System. The user has three questions: core architecture, essential tools, and implementing the workflow.

First, I need to understand what a Multi-Agent Deep Research System is. From what I know, multi-agent systems involve multiple interacting agents, and deep research might relate to using deep learning techniques. So the core architecture probably involves agent communication, coordination mechanisms, and integration w

In [ ]:
from tools.web_operations import scrape_webpages
import json

summarization_agent = create_agent("summarization_agent", "qwen3:14b", [scrape_webpages])
content = read_file.invoke('prompts/search_agent_output_example.json')
# Convert string to json object
content = json.loads(content)

for question in content['questions']:
    input_message = {"role": "user", "content": json.dumps(question),}

    # Use the agent
    config = {"configurable": {"thread_id": "abc123"}, "recursion_limit": 10, "callbacks": [langfuse_handler]}
    for step in summarization_agent.stream(
        {"messages": [input_message]}, config, stream_mode="values"
    ):
        step["messages"][-1].pretty_print()

<div class="admonition tip">
    <p class="admonition-title">Set up <a href="https://smith.langchain.com">LangSmith</a> for LangGraph development</p>
    <p style="padding-top: 5px;">
        Sign up for LangSmith to quickly spot issues and improve the performance of your LangGraph projects. LangSmith lets you use trace data to debug, test, and monitor your LLM apps built with LangGraph — read more about how to get started <a href="https://docs.smith.langchain.com">here</a>. 
    </p>
</div>

## Create Tools

Each team will be composed of one or more agents each with one or more tools. Below, define all the tools to be used by your different teams.

We'll start with the research team.

**ResearchTeam tools**

The research team can use a search engine and url scraper to find information on the web. Feel free to add additional functionality below to boost the team performance!

In [ ]:

from web_scraper import scrape_webpages


**Document writing team tools**

Next up, we will give some tools for the doc writing team to use.
We define some bare-bones file-access tools below.

Note that this gives the agents access to your file-system, which can be unsafe. We also haven't optimized the tool descriptions for performance.

## Helper Utilities

We are going to create a few utility functions to make it more concise when we want to:

1. Create a worker agent.
2. Create a supervisor for the sub-graph.

These will simplify the graph compositional code at the end for us so it's easier to see what's going on.

In [ ]:
from typing import List, Optional, Literal
from langchain_core.language_models.chat_models import BaseChatModel

from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.types import Command
from langchain_core.messages import HumanMessage, trim_messages


class State(MessagesState):
    next: str


def make_supervisor_node(llm: BaseChatModel, members: list[str]) -> str:
    options = ["FINISH"] + members
    system_prompt = (
        "You are a supervisor tasked with managing a conversation between the"
        f" following workers: {members}. Given the following user request,"
        " respond with the worker to act next. Each worker will perform a"
        " task and respond with their results and status. When finished,"
        " respond with FINISH."
    )

    class Router(TypedDict):
        """Worker to route to next. If no workers needed, route to FINISH."""

        next: Literal[*options]

    def supervisor_node(state: State) -> Command[Literal[*members, "__end__"]]:
        """An LLM-based router."""
        messages = [
            {"role": "system", "content": system_prompt},
        ] + state["messages"]
        response = llm.with_structured_output(Router).invoke(messages)
        goto = response["next"]
        if goto == "FINISH":
            goto = END

        return Command(goto=goto, update={"next": goto})

    return supervisor_node

## Define Agent Teams

Now we can get to define our hierarchical teams. "Choose your player!"

### Research Team

The research team will have a search agent and a web scraping "research_agent" as the two worker nodes. Let's create those, as well as the team supervisor.

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_ollama import ChatOllama
from langgraph.prebuilt import create_react_agent

llm = ChatOllama(model="qwen3")

search_agent = create_react_agent(llm, tools=[tavily_tool])


def search_node(state: State) -> Command[Literal["supervisor"]]:
    result = search_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="search")
            ]
        },
        # We want our workers to ALWAYS "report back" to the supervisor when done
        goto="supervisor",
    )


web_scraper_agent = create_react_agent(llm, tools=[scrape_webpages])


def web_scraper_node(state: State) -> Command[Literal["supervisor"]]:
    result = web_scraper_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="web_scraper")
            ]
        },
        # We want our workers to ALWAYS "report back" to the supervisor when done
        goto="supervisor",
    )


research_supervisor_node = make_supervisor_node(llm, ["search", "web_scraper"])

Now that we've created the necessary components, defining their interactions is easy. Add the nodes to the team graph, and define the edges, which determine the transition criteria.

In [ ]:
research_builder = StateGraph(State)
research_builder.add_node("supervisor", research_supervisor_node)
research_builder.add_node("search", search_node)
research_builder.add_node("web_scraper", web_scraper_node)

research_builder.add_edge(START, "supervisor")
research_graph = research_builder.compile()

In [ ]:
from IPython.display import Image, display

display(Image(research_graph.get_graph().draw_mermaid_png()))

We can give this team work directly. Try it out below.

In [ ]:
# for s in research_graph.stream(
#     {"messages": [("user", "when is Ed Sheeran's next tour in 2025?")]},
#     config={"recursion_limit": 100, "callbacks": [langfuse_handler]},
#     ):
#     print(s)
#     print("---")

### Document Writing Team

Create the document writing team below using a similar approach. This time, we will give each agent access to different file-writing tools.

Note that we are giving file-system access to our agent here, which is not safe in all cases.

In [ ]:
llm = ChatOllama(model="qwen3")

doc_writer_agent = create_react_agent(
    llm,
    tools=[write_file, edit_document, read_file],
    prompt=(
        "You can read, write and edit documents based on note-taker's outlines. "
        "Don't ask follow-up questions."
    ),
)


def doc_writing_node(state: State) -> Command[Literal["supervisor"]]:
    result = doc_writer_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="doc_writer")
            ]
        },
        # We want our workers to ALWAYS "report back" to the supervisor when done
        goto="supervisor",
    )


note_taking_agent = create_react_agent(
    llm,
    tools=[create_outline, read_file],
    prompt=(
        "You can read documents and create outlines for the document writer. "
        "Don't ask follow-up questions."
    ),
)


def note_taking_node(state: State) -> Command[Literal["supervisor"]]:
    result = note_taking_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="note_taker")
            ]
        },
        # We want our workers to ALWAYS "report back" to the supervisor when done
        goto="supervisor",
    )


chart_generating_agent = create_react_agent(
    llm, tools=[read_file, python_repl_tool]
)


def chart_generating_node(state: State) -> Command[Literal["supervisor"]]:
    result = chart_generating_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(
                    content=result["messages"][-1].content, name="chart_generator"
                )
            ]
        },
        # We want our workers to ALWAYS "report back" to the supervisor when done
        goto="supervisor",
    )


doc_writing_supervisor_node = make_supervisor_node(
    llm, ["doc_writer", "note_taker", "chart_generator"]
)

With the objects themselves created, we can form the graph.

In [ ]:
# Create the graph here
paper_writing_builder = StateGraph(State)
paper_writing_builder.add_node("supervisor", doc_writing_supervisor_node)
paper_writing_builder.add_node("doc_writer", doc_writing_node)
paper_writing_builder.add_node("note_taker", note_taking_node)
paper_writing_builder.add_node("chart_generator", chart_generating_node)

paper_writing_builder.add_edge(START, "supervisor")
paper_writing_graph = paper_writing_builder.compile()

In [ ]:
from IPython.display import Image, display

display(Image(paper_writing_graph.get_graph().draw_mermaid_png()))

In [ ]:
# for s in paper_writing_graph.stream(
#     {
#         "messages": [
#             (
#                 "user",
#                 "Write an outline for poem about cats and then write the poem to disk.",
#             )
#         ]
#     },
#     {"recursion_limit": 100, "callbacks": [langfuse_handler]},
# ):
#     print(s)
#     print("---")

## Add Layers

In this design, we are enforcing a top-down planning policy. We've created two graphs already, but we have to decide how to route work between the two.

We'll create a _third_ graph to orchestrate the previous two, and add some connectors to define how this top-level state is shared between the different graphs.

In [ ]:
from langchain_core.messages import BaseMessage

teams_supervisor_node = make_supervisor_node(llm, ["research_team", "writing_team"])

In [ ]:
def call_research_team(state: State) -> Command[Literal["supervisor"]]:
    response = research_graph.invoke({"messages": state["messages"][-1]})
    return Command(
        update={
            "messages": [
                HumanMessage(
                    content=response["messages"][-1].content, name="research_team"
                )
            ]
        },
        goto="supervisor",
    )


def call_paper_writing_team(state: State) -> Command[Literal["supervisor"]]:
    response = paper_writing_graph.invoke({"messages": state["messages"][-1]})
    return Command(
        update={
            "messages": [
                HumanMessage(
                    content=response["messages"][-1].content, name="writing_team"
                )
            ]
        },
        goto="supervisor",
    )


# Define the graph.
super_builder = StateGraph(State)
super_builder.add_node("supervisor", teams_supervisor_node)
super_builder.add_node("research_team", call_research_team)
super_builder.add_node("writing_team", call_paper_writing_team)

super_builder.add_edge(START, "supervisor")
super_graph = super_builder.compile()

In [ ]:
from IPython.display import Image, display

display(Image(super_graph.get_graph().draw_mermaid_png()))

In [ ]:
for s in super_graph.stream(
    {
        "messages": [
            ("user", "Research AI agents and write in a file using the writing_team a brief report about them.")
        ],
    },
    {"recursion_limit": 150, "callbacks": [langfuse_handler]},
):
    print(s)
    print("---")